## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-21-10-43-01 +0000,nyt,Protesters in Manila Accuse Government of Misu...,https://www.nytimes.com/2025/09/21/world/asia/...
1,2025-09-21-10-31-32 +0000,bbc,"'I spent $8,000 to get back to US' after Trump...",https://www.bbc.com/news/articles/cn4wwwz2p1po...
2,2025-09-21-10-25-40 +0000,nyt,Trump’s Tariffs on Switzerland Prompt Identity...,https://www.nytimes.com/2025/09/21/business/sw...
3,2025-09-21-10-21-31 +0000,bbc,Heathrow braces for second day of disruption a...,https://www.bbc.com/news/articles/cwy88857llno...
4,2025-09-21-10-13-45 +0000,nypost,Alleged New Hampshire country club gunman Hunt...,https://nypost.com/2025/09/21/us-news/alleged-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2355/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,33
141,kirk,16
140,charlie,13
25,new,11
9,visa,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
125,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...,76
93,2025-09-20-23-11-48 +0000,cbc,White House scrambles to clarify Trump plan to...,https://www.cbc.ca/news/world/trump-travel-ban...,65
64,2025-09-21-02-03-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...,64
123,2025-09-20-20-04-18 +0000,nypost,White House tries easing panic over high-skill...,https://nypost.com/2025/09/20/us-news/white-ho...,63
33,2025-09-21-09-00-00 +0000,wsj,Some Republicans in the House and Senate who s...,https://www.wsj.com/politics/policy/republican...,62


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
125,76,2025-09-20-19-50-05 +0000,nypost,White House rips Barack Obama for suggesting T...,https://nypost.com/2025/09/20/us-news/white-ho...
79,50,2025-09-21-00-48-31 +0000,nyt,Arrest in Arizona Before Charlie Kirk Memorial...,https://www.nytimes.com/2025/09/20/us/charlie-...
123,50,2025-09-20-20-04-18 +0000,nypost,White House tries easing panic over high-skill...,https://nypost.com/2025/09/20/us-news/white-ho...
111,35,2025-09-20-21-33-56 +0000,nypost,High-speed police chase ends in deadly shootou...,https://nypost.com/2025/09/20/us-news/massive-...
4,34,2025-09-21-10-13-45 +0000,nypost,Alleged New Hampshire country club gunman Hunt...,https://nypost.com/2025/09/21/us-news/alleged-...
164,32,2025-09-20-15-22-28 +0000,nypost,Q&A: NYC Mayor Eric Adams calls Jews his secre...,https://nypost.com/2025/09/20/us-news/q-amp-a-...
64,31,2025-09-21-02-03-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
167,25,2025-09-20-14-54-49 +0000,nypost,Trump says he axed US attorney investigating N...,https://nypost.com/2025/09/20/us-news/trump-sa...
174,24,2025-09-20-13-31-27 +0000,nypost,US ambassador to Canada rips ‘anti-American’ s...,https://nypost.com/2025/09/20/world-news/us-am...
52,23,2025-09-21-05-01-20 +0000,bbc,"Israel will occupy more West Bank land, but re...",https://www.bbc.com/news/articles/c0ez9qxzl2jo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
